face blur by haarcascade

In [1]:
# blur thr face

import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.GaussianBlur(face, (105,105), 0)
        frame[y:y+h, x:x+w] = face

    cv2.imshow("Blurred Face", frame)
    key = cv2.waitKey(1)
    if key == ord("q") or key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

Face blur by mediapipe

In [2]:
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection

cap = cv2.VideoCapture(0)

    
# Initiate mediapipe face detection
with mp_face_detection.FaceDetection(min_detection_confidence=0.2) as face_detection:
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()

        # Convert the image to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image with mediapipe
        results = face_detection.process(frame)

        # Draw face detection landmarks on the frame
        if results.detections:
            for detection in results.detections:
                # Get the bounding box of the face
                bbox = detection.location_data.relative_bounding_box
                h, w, c = frame.shape
                bbox_x = int(bbox.xmin * w)
                bbox_y = int(bbox.ymin * h)
                bbox_w = int(bbox.width * w)
                bbox_h = int(bbox.height * h)

                # Draw the bounding box on the frame
                cv2.rectangle(frame, (bbox_x, bbox_y), (bbox_x + bbox_w, bbox_y + bbox_h), (150, 170, 150), 2)

                # Apply Gaussian blur to the face
                face = frame[bbox_y:bbox_y+bbox_h, bbox_x:bbox_x+bbox_w]
                face = cv2.GaussianBlur(face, (105, 105), 0)
                frame[bbox_y:bbox_y+bbox_h, bbox_x:bbox_x+bbox_w] = face

        # Show the processed image
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        cv2.imshow('frame', frame)

        # Exit on 'q' key 'esc'
        key = cv2.waitKey(1)
        if key == ord("q") or key == 27 :
            break

# Release the video capture
cap.release()

# Close the window
cv2.destroyAllWindows()

Using TensorFlow backend.


Background blur

In [3]:
import cv2
import mediapipe as mp
import numpy as np

mp_selfie_segmentation = mp.solutions.selfie_segmentation

cap = cv2.VideoCapture(0)

# Initiate mediapipe selfie segmentation
with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as selfie_segmentation:
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
        
        # Calculate the fps
        #curr_time = time.time()
        #fps = 1 / (curr_time - prev_time)
        #prev_time = curr_time

        # Convert the image to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image with mediapipe selfie segmentation
        frame.flags.writeable = False
        results = selfie_segmentation.process(frame)
        frame.flags.writeable = True

        # Create a mask for the segmented image
        condition = np.stack((results.segmentation_mask,)*3, axis=-1) > 0.1

        # Apply Gaussian blur to the background image
        bg_image = cv2.GaussianBlur(frame, (55, 55), 0)

        # Create the final image by blending the original and blurred images
        image = np.where(condition, frame, bg_image)

        # Convert the image back to BGR for display
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Show the processed image
        #cv2.putText(image, f"FPS: {int(fps)}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('image', image)

        # Exit on 'q' key or 'esc' key
        key = cv2.waitKey(1)
        if key == ord("q") or key == 27 :
            break

# Release the video capture
cap.release()

# Close the window
cv2.destroyAllWindows()
